In [1]:
using<-function(...) {
    libs<-unlist(list(...))
    req<-unlist(lapply(libs,require,character.only=TRUE))
    need<-libs[req==FALSE]
    if(length(need)>0){ 
        install.packages(need)
        lapply(need,require,character.only=TRUE)
    }
}

In [2]:
using("data.table", "tidyverse", "naniar", "stringr", "readr",  "dplyr", "magrittr", "readxl", "writexl", "sjmisc", "tidyr", "rdflib"
                  )


Loading required package: data.table

Loading required package: tidyverse

-- Attaching core tidyverse packages ------------------------ tidyverse 2.0.0 --
v dplyr     1.1.0     v readr     2.1.4
v forcats   1.0.0     v stringr   1.5.0
v ggplot2   3.4.1     v tibble    3.2.0
v lubridate 1.9.2     v tidyr     1.3.0
v purrr     1.0.1     
-- Conflicts ------------------------------------------ tidyverse_conflicts() --
x dplyr::between()     masks data.table::between()
x dplyr::filter()      masks stats::filter()
x dplyr::first()       masks data.table::first()
x lubridate::hour()    masks data.table::hour()
x lubridate::isoweek() masks data.table::isoweek()
x dplyr::lag()         masks stats::lag()
x dplyr::last()        masks data.table::last()
x lubridate::mday()    masks data.table::mday()
x lubridate::minute()  masks data.table::minute()
x lubridate::month()   masks data.table::month()
x lubridate::quarter() masks data.table::quarter()
x lubridate::second()  masks data.table::second(

In [3]:
barometer_dt_raw <- readxl::read_excel("DGZ/DECIDE_MTA_UGENT_14nov2022.xlsx")
barometer_aero_cult_raw <- readxl::read_excel("DGZ/DECIDE_MTA_UGENT_BAC_AERO_14nov2022.xlsx")
barometer_myco_cult_raw <- readxl::read_excel("DGZ/DECIDE_MTA_UGENTBAC_MYCO_14nov2022.xlsx")




In [4]:
## Data manipulation AEROBIC CULTURE results

barometer_aero_cult <- barometer_aero_cult_raw %>%
  dplyr::rename(
    Filenumber = Dossiernummer,
    Pathogen_identification = 'KIEMSTAAL IDENTIFICATIE',
    Pathogen_result = 'KIEMSTAAL RESULTAAT',
    Samplenumber = 'Staalnummer', 
  )%>%
  
  dplyr::mutate(
    Parameter_code = 'BAC_AERO',
    Result = 'OK'
  )%>%
  dplyr::select(
    Filenumber,
    Pathogen_identification,
    Pathogen_result,
    Parameter_code,
    Samplenumber,
    Result)  %>%
  dplyr::filter(
    Pathogen_identification %in% c("Pasteurella multocida","Mannheimia haemolytica","Histophilus somni","Mycoplasma bovis")
  ) %>%
  dplyr::distinct()
#show(barometer_aero_cult)

In [5]:
##Intermediate table is needed

df_samples <- data.frame(
  Result = c('OK', 'OK', 'OK', 'OK'),
  Parameter_code = c('BAC_AERO','BAC_AERO','BAC_AERO', 'BAC_MYCOPLASMA'),
  Diagnostic_test= c('Culture','Culture','Culture', 'Culture'),
  Pathogen_identification=c("Pasteurella multocida","Mannheimia haemolytica","Histophilus somni", 'Mycoplasma bovis')
)
#print(df_samples)

In [6]:
## Data manipulation MYCOPLASMA CULTURE results

barometer_myco_cult <- barometer_myco_cult_raw %>%
  dplyr::rename(
    Filenumber = Dossiernummer,
    Pathogen_identification = 'KIEMSTAAL IDENTIFICATIE',
    Mycoplasma_result = 'KIEMSTAAL RESULTAAT',
    Samplenumber = 'Staalnummer'
  )%>%
  dplyr::mutate(
    Parameter_code = 'BAC_MYCOPLASMA',
    Result = 'OK'
  ) %>%
  dplyr::select(
    Filenumber,
    Pathogen_identification,
    Mycoplasma_result,
    Parameter_code,
    Samplenumber,
    Result)  %>%
  dplyr::filter(
    Pathogen_identification %in% c("Mycoplasma bovis")
  ) %>%
  dplyr::distinct()

#print(barometer_myco_cult)

In [7]:
## Data manipulation PCR results

barometer_dt <- barometer_dt_raw %>% 
  dplyr::rename(
    Filenumber=Dossiernummer, 
    Samplenumber = Staalnummer,
    Sample_type = Staaltype,
    Parameter_code = PARAMETER_CODE,
    Pathogen = Onderzoek,
    Result = Resultaat,
    Date = Creatiedatum,
    Postal_code = Postcode, 
    Farm_ID = ANON_ID
    ) %>%
  dplyr::mutate(
    Country='Belgium',
    Diagnostic_test = case_when(
      Parameter_code %in% c('BAC_AERO','BAC_MYCOPLASMA') ~ 'Culture',
      TRUE ~ 'PCR'
    ),
    Lab_reference='1', 
    Sample_type = case_when(
      Sample_type == "RU Broncho-alveolar lavage (BAL)" ~ 'BAL', 
      Sample_type == "RU Anderen" ~'Unknown',
      Sample_type %in% c("RU Swabs", "RU Swab", 'RU Neusswab', 'RU Neusswabs') ~ 'Swab',
      Sample_type %in% c("RU Kadaver", "RU Organen") ~ 'Autopsy',
      TRUE ~ 'Missing'
    ),
    Breed = case_when(
      Bedrijfstype == 'VCALF' ~ 'Veal',
      is.na(MEAT) ~ 'Unknown',
      (as.numeric(MEAT)/as.numeric(TOTAL))>0.9 ~ 'Beef', 
      (as.numeric(MILK)/as.numeric(TOTAL))>0.9 ~ 'Dairy',
      TRUE ~ 'Mixed' 
    ),
    Pathogen = case_when(
      Pathogen %in% c(
        "AD Pasteurella multocida Ag (PCR)", 
        "AD Pasteurella multocida Ag pool (PCR)", 
        "AD P. multocida Ag (PCR)", 
        "AD P. multocida Ag pool (PCR)") ~ 'Pasteurella multocida',
      Pathogen %in% c(
        "AD Mannheimia haemolytica Ag (PCR)", 
        "AD Mannheimia haemolytica Ag pool (PCR)") ~ 'Mannheimia haemolytica',
      Pathogen %in% c("RU PI3 Ag (PCR)", "RU PI3 Ag pool (PCR)") ~ 'PI3',
      Pathogen %in% c("RU BRSV Ag (PCR)", "RU BRSV Ag pool (PCR)") ~ 'BRSV',
      Pathogen %in% c(
        "AD Histophilus somnus (PCR)", 
        "AD Histophilus somnus Ag (PCR)", 
        "AD Histophilus somnus Ag pool (PCR)", 
        "AD Histophilus somni Ag (PCR)",
        "AD Histophilus somni Ag pool (PCR)") ~ 'Histophilus somni',
      Pathogen %in% c(
        "RU Mycoplasma bovis (PCR)", 
        "RU Mycoplasma bovis Ag pool (PCR)", 
        "RU Mycoplasma bovis Ag (PCR)") ~ 'Mycoplasma bovis',
      Pathogen %in% c("AD Corona Ag (PCR)", "AD Corona Ag pool (PCR)") ~ 'BCV'
    ),
      Province = case_when(
        between(as.numeric(Postal_code), 1000, 1299) ~ 'Brussels',
        between(as.numeric(Postal_code), 1300, 1499) ~ 'Walloon Brabant',
        between(as.numeric(Postal_code), 1500, 1999) ~ 'Flemish Brabant',
        between(as.numeric(Postal_code), 3000, 3499) ~ 'Antwerp',
        between(as.numeric(Postal_code), 2000, 2999) ~ 'Limburg',
        between(as.numeric(Postal_code), 3500, 3999) ~ 'Limburg',
        between(as.numeric(Postal_code), 4000, 4999) ~ 'Liège',
        between(as.numeric(Postal_code), 5000, 5999) ~ 'Namur',
        between(as.numeric(Postal_code), 6000, 6599) ~ 'Hainaut',
        between(as.numeric(Postal_code), 7000, 7999) ~ 'Hainaut',
        between(as.numeric(Postal_code), 6600, 6999) ~ 'Luxembourg',
        between(as.numeric(Postal_code), 8000, 8999) ~ 'West Flanders',
        TRUE ~ 'East Flanders'
            )
  
  )%>%
  dplyr::select(
    Filenumber,
    Diagnostic_test,
    Samplenumber,
    Country,
    Lab_reference,
    Sample_type,
    Breed,
    Parameter_code,
    Result,
    Pathogen,
    Date,
    Postal_code,
    Province,
    Farm_ID
  ) %>%
  dplyr::distinct()

#print(barometer_dt)


In [8]:
## Join all three files

barometer <-
  barometer_dt %>%
  dplyr::left_join(df_samples, by = c('Diagnostic_test','Result', 'Parameter_code')) %>%
  dplyr::left_join(
      barometer_aero_cult, by = c('Filenumber', 'Samplenumber', 'Result', 'Parameter_code', 'Pathogen_identification')
      ) %>%
  dplyr::left_join(
      barometer_myco_cult, by = c('Filenumber', 'Samplenumber', 'Result', 'Parameter_code', 'Pathogen_identification')
  ) %>%
  dplyr::mutate(
      Pathogen = case_when(
      (Pathogen == 'Pasteurella multocida') ~ 'PM',
      (Pathogen == 'Histophilus somni') ~ 'HS',
      (Pathogen == 'Mannheimia haemolytica') ~ 'MH',
      (Pathogen == 'Mycoplasma bovis') ~ 'MB',
      TRUE ~ Pathogen
    ),
    Pathogen = case_when(
      (Pathogen_identification == 'Pasteurella multocida') ~ 'PM',
      (Pathogen_identification == 'Histophilus somni') ~ 'HS',
      (Pathogen_identification == 'Mannheimia haemolytica') ~ 'MH',
      (Pathogen_identification == 'Mycoplasma bovis') ~ 'MB',
      TRUE ~ Pathogen
    ),
      Result = case_when(
        Result %in% c("Twijfelachtig (PCR)", "POSITIEF", "GEDETECTEERD", "GEDETECTEERD (sterk)", "GEDETECTEERD (zwak)",
        "GEDETECTEERD (matig)","GEDETECTEERD (zeer sterk)", "GEDETECTEERD (zeer zwak)") ~ 1,
        Result %in% c("negatief", "Niet gedetecteerd") ~ 0,
        Result %in% c("NI", "niet interpreteerbaar", "Inhibitie") ~ as.numeric(NA),
      Parameter_code == 'BAC_AERO' & is.na(Pathogen_result) ~ 0,
      Parameter_code == 'BAC_AERO' & !is.na(Pathogen_result) ~ 1, 
      Parameter_code == 'BAC_MYCOPLASMA' & is.na(Mycoplasma_result) ~ as.numeric(NA),
      Parameter_code == 'BAC_MYCOPLASMA' & Mycoplasma_result == 'neg' ~ 0,
      Parameter_code == 'BAC_MYCOPLASMA' & sjmisc::str_contains(Mycoplasma_result, 'POS') ~ 1,
      TRUE ~ as.numeric(NA)
    )

  )

Warning message in dplyr::left_join(., df_samples, by = c("Diagnostic_test", "Result", :
"Each row in `x` is expected to match at most 1 row in `y`.
i Row 1693 of `x` matches multiple rows.
i If multiple matches are expected, set `multiple = "all"` to silence this
  warning."
Warning message in dplyr::left_join(., barometer_myco_cult, by = c("Filenumber", :
"Each row in `x` is expected to match at most 1 row in `y`.
i Row 2069 of `x` matches multiple rows.
i If multiple matches are expected, set `multiple = "all"` to silence this
  warning."


In [9]:
library(magrittr)
library(rdflib)
library(XML)

In [ ]:
# create an RDF object
Literal <- function(value, datatype="xsd:string") {
  rdf_literal(value, datatype=datatype)
}
rdf <- rdf()
 
# create a namespace prefix for the ontology
onto <- "http://www.purl.org/decide/LivestockHealthOnto"
 
# iterate through each row in the barometer DataFrame
for (i in 1:nrow(barometer)) {
  row <- barometer[i,] 
# Anonymize sample number and file number
anonymizedSamplenumber <- paste0("Sample_", digest::digest(as.character(row$Samplenumber), algo = "sha256"))
anonymizedFilenumber <- paste0("File_", digest::digest(as.character(row$Filenumber), algo = "sha256"))

# create a URI for the CattleSample individual based on the row ID
CattleSample <- paste0(prefix, "CattleSample", i)

# add triples to the RDF graph
 rdf_add(rdf, CattleSample, paste0(onto, "hasFileNumber"), anonymizedFilenumber, datatype = "xsd:string")
 rdf_add(rdf, CattleSample, paste0(onto, "hasSampleNumber"), anonymizedSamplenumber, datatype = "xsd:string")
  rdf_add(rdf, CattleSample, paste0(onto, "hasResult"), as.character(row[3]), datatype="xsd:string")
  rdf_add(rdf, CattleSample, paste0(onto, "hasParametercode"), as.character(row[4]), datatype="xsd:string")
  rdf_add(rdf, CattleSample, paste0(onto, "hasPathogenIdentification"), as.character(row[5]), datatype="xsd:string")
  rdf_add(rdf, CattleSample, paste0(onto, "hasDiagnosticTest"), as.character(row[6]), datatype="xsd:string")
  rdf_add(rdf, CattleSample, paste0(onto, "hasCountry"), as.character(row[7]), datatype="xsd:string")
  rdf_add(rdf, CattleSample, paste0(onto, "hasLabRefference"), as.character(row[8]), datatype="xsd:string")
  rdf_add(rdf, CattleSample, paste0(onto, "hasSampleType"), as.character(row[9]), datatype="xsd:string")
  rdf_add(rdf, CattleSample, paste0(onto, "hasBreed"), as.character(row[10]), datatype="xsd:string")
  rdf_add(rdf, CattleSample, paste0(onto, "hasPathogen"), as.character(row[11]), datatype="xsd:string")
  rdf_add(rdf, CattleSample, paste0(onto, "hasDate"), as.character(row[12]), datatype="xsd:string")
  rdf_add(rdf, CattleSample, paste0(onto, "hasPostalCode"), as.character(row[13]), datatype="xsd:string")
  rdf_add(rdf, CattleSample, paste0(onto, "hasProvince"), as.character(row[14]), datatype="xsd:string")
  rdf_add(rdf, CattleSample, paste0(onto, "hasFarmIdentification"), as.character(row[15]), datatype="xsd:string")
  rdf_add(rdf, CattleSample, paste0(onto, "hasPathogenResult"), as.character(row[16]), datatype="xsd:string")
  rdf_add(rdf, CattleSample, paste0(onto, "hasMicoplasmaResult"), as.character(row[17]), datatype="xsd:string")

}
 
# print the RDF graph (for testing)
cat(rdf_serialize(rdf, format="turtle"))
 
# output the RDF graph to a file (replace with your desired filename)
rdf_serialize(rdf, "output/RDFoutputCattleSample.ttl", format="turtle")